In [1]:
%load_ext autoreload
%autoreload 2

In [50]:
import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True
from unsloth import FastLanguageModel, unsloth_train

In [40]:
import sys
import os

import pandas as pd

# Add the parent directory of src to the path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.db.manager import DBManager

In [3]:
if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
# attn_implementation = "eager"
print(f"Using {attn_implementation} for attention computation.")
# QLora?

Using flash_attention_2 for attention computation.


In [4]:
model_id = 'defog/llama-3-sqlcoder-8b'
model_dir = f"/model/{model_id.replace('/', '-')}"

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    # max_seq_length = max_seq_length,
    dtype = torch_dtype,
    load_in_4bit = False,
    load_in_8bit = False,
    # quantization_config=BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_quant_type="nf4",
    #     bnb_4bit_compute_dtype=torch_dtype
    #     # load_in_8bit=True,
    #     # llm_int8_enable_fp32_cpu_offload=False if not "27B" in model_id else True,
    # ),
    # device_map=device,
    cache_dir=f"{model_dir}/cache",
    attn_implementation=attn_implementation,
    # trust_remote_code=True
    # local_files_only=True
)
tokenizer.padding_side = "left"
# tokenizer.truncation_side = "left"
print(f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}")
print(f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}")

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.3.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


defog/llama-3-sqlcoder-8b does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.
Pad Token id: 128255 and Pad Token: <|reserved_special_token_250|>
EOS Token id: 128009 and EOS Token: <|eot_id|>


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    

In [6]:
prompt = f"""주어진 테이블 스키마를 참고해 질문에 답하기 위한 쿼리를 작성해줘
데이터베이스: timescale db (postgresql 기반)

CREATE TABLE IF NOT EXISTS public.idu_t
(
    id integer NOT NULL DEFAULT nextval('idu_t_id_seq'::regclass),
    name character varying(50) COLLATE pg_catalog."default",
    CONSTRAINT idu_t_pkey PRIMARY KEY (id)
)

CREATE TABLE IF NOT EXISTS public.data_t
(
    id integer NOT NULL DEFAULT nextval('data_t_id_seq'::regclass),
    idu_id integer,
    roomtemp double precision,
    settemp double precision,
    "timestamp" timestamp without time zone NOT NULL
)

조회할때 NaN인 row는 제외

지금은 2022-09-30 12:00:00 입니다.
"""


In [48]:
import re
def extract_content(text: str):
    """Extract content from model output."""
    if "start_header_id" in text:
        pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else None


def generate(query):
    """
    suppress all outputs, warnings, errors from this function
    모든 출력, 경고, 에러를 억제하는 함수입니다.
    """
    import sys
    import contextlib
    import warnings
    import io

    # Suppress stdout, stderr, and warnings
    with contextlib.redirect_stdout(io.StringIO()), \
         contextlib.redirect_stderr(io.StringIO()), \
         warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            chat = [
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"Input:{query};"},
            ]

            chat = tokenizer.apply_chat_template(
                chat,
                tokenize=False,
                add_generation_prompt=True,
                # return_tensors="pt"
            )

            chat = re.sub(
                r'(\nCutting Knowledge Date:.*?\nToday Date:.*?\n\n)', 
                '', 
                chat
            )

            # Tokenize the chat input using the tokenizer
            # 토크나이저를 사용하여 chat 입력을 토크나이즈합니다.
            input_ids = tokenizer(
                chat, 
                return_tensors="pt"
            ).input_ids.to(model.device)

            outputs = model.generate(
                    input_ids=input_ids,
                    max_new_tokens=1000,
                    temperature=0.001,
                    pad_token_id=tokenizer.pad_token_id,
                )
            decoded = tokenizer.batch_decode(
                outputs, 
                skip_special_tokens=False
            )

            return extract_content(decoded[0])
        except Exception:
            # suppress all errors, return None
            # 모든 에러를 억제하고 None을 반환합니다.
            return None


    

In [54]:
queries = [
    # "지금 idu=idu_name=01_IB7의 roomtemp 알려줘",
    # "지금 idu=idu_name=01_IB7의 roomtemp와 settemp 알려줘",
    # "지난주 수요일 idu=idu_name=01_IB7의 roomtemp 알려줘",
    # "오늘하루 idu=idu_name=01_IB5의 roomtemp 알려줘",
    # "오늘중 idu='idu_name=01_IB5'의 roomtemp와 idu='idu_name=01_IB7'의 settemp가 같은 데이터",
    "어제 idu_name=01_IB5과 idu_name=01_IB7의 settemp 차이 알려줘", # X
    "오늘 idu_name=01_IB5과 idu_name=01_IB7의 평균 roomtemp차이 알려줘",
    "올해 여름 idu_name=02_I81 평균roomtemp 알려줘",
    "올해 봄 idu_name=01_IB7 제일 추웠던 날 알려줘",
    "4월 idu_name=02_I81 평균roomtemp 알려줘",
    "이번달 중 idu_name=01_IB5 roomtemp가 가장 덜 더운날이 언제야?",
    "2주전 idu_name=01_IB5과 idu_name=01_IB7 합쳐서 settemp가 가장 낮은날이 언제야?",
    "idu_name=01_IB5의 현재 settemp 알려줘",
    "8일전 settemp는?",
    "10년 전 오늘 idu_name=01_IB5 roomtemp는?",
    "idu_name=01_IB5, idu_name=01_IB7, idu_name=02_I81 중 가장 추운 방은?",
    "지난달 오늘 오후 2시에 idu_name=01_IB7의 settemp는 어땠어?",
    "idu_name=01_IB7의 현재 roomtemp랑 settemp 알려줘",
    "이번주 idu_name=01_IB5 평균 roomtemp 알려줘",
    "지난달 settemp 평균을 알려줘.",
    "idu_name=01_IB5 이번달 제일 추웠던 날은 언제냐?",
    "작년 idu_name=01_IB7 가장 더웠던 달은?",
    "오늘 오전 11시에 idu_name=01_IB7의 roomtemp는 어땠어?",
    "오늘 오후 4시부터 6시까지 roomtemp 평균 알려줘",
    "지난주에 settemp와 roomtemp 차이가 가장 많이 났던 날은?",
    "idu_name=01_IB5과 idu_name=01_IB7중 더 추운곳은 어디야?",
    "이번주 idu_name=01_IB5과 idu_name=02_I81의 평균 roomtemp 알려줘",
    "현재 settemp랑 roomtemp 차이 알려줘.",
    "지난달에 settemp와 roomtemp 차이가 가장 많이 났던 날은?",
    "이번주 idu_name=01_IB5과 idu_name=01_IB7의 평균 roomtemp 차이 알려줘",
    "2주전 가장 더웠던 날 알려줘",
    "idu_name=01_IB7 습도 알려줘",
    "지난 3일 동안 idu_name=01_IB5 실내 roomtemp 평균 값 알려줘.",
    "오늘 오후 5시에 idu_name=01_IB7의 settemp는 어땠어?",
    "올해 여름 idu_name=01_IB5 roomtemp 최대값과 최소값 알려줘",
    "idu_name=01_IB5과 idu_name=02_I81 중 가장 더운 방은?",
]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


An error occurred while executing SQL query: more than one row returned by a subquery used as an expression

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Error: 'NoneType' object is not iterable


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Query: 올해 봄 idu_name=01_IB7 제일 추웠던 날 알려줘
Result: SELECT d.idu_id, d.roomtemp FROM public.data_t d JOIN public.idu_t i ON d.idu_id = i.id WHERE i.name = '01_IB7' AND d."timestamp" BETWEEN '2022-03-01' AND '2022-06-30' AND d.roomtemp IS NOT NULL ORDER BY d.roomtemp ASC LIMIT 1;

results [(2, -1.0)]


...

Query: 현재 settemp랑 roomtemp 차이 알려줘.
Result: SELECT (s.settemp - r.roomtemp) AS temp_diff FROM public.data_t s JOIN public.data_t r ON s.id = r.id WHERE s."timestamp" = '2022-09-30 12:00:00' AND s.settemp IS NOT NULL AND r.roomtemp IS NOT NULL;

results [(-5.0,), (-4.0,), (-3.0,), (-3.0,), (-3.0,), (-3.0,), (-3.5,)]


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Query: 지난달에 settemp와 roomtemp 차이가 가장 많이 났던 날은?
Result: SELECT d."timestamp" FROM public.data_t d WHERE d."timestamp" BETWEEN (TIMESTAMP '2022-09-30 12:00:00' - interval '1 month') AND TIMESTAMP '2022-09-30 12:00:00' AND d.settemp IS NOT NULL AND d.roomtemp IS NOT NULL ORDER BY (d.settemp - d.roomtemp) DESC LIMIT 1;

results [(datetime.datetime(2022, 9, 29, 6, 27),)]


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{'success': 15, 'error': 1, 'nan': 16}


In [ ]:
prompt += """
실내온도: roomtemp
설정온도: settemp
우리반: 01_IB5
옆반: 01_IB7
앞반: 02_I81
"""

queries = [
    "어제 우리반과 옆반의 설정온도 차이 알려줘", # X
    "오늘 우리반과 옆반의 평균 실내온도차이 알려줘",
    "올해 여름 앞반 평균실내온도 알려줘",
    "올해 봄 옆반 제일 추웠던 날 알려줘",
    "4월 앞반 평균실내온도 알려줘",
    "이번달 중 우리반 실내온도가 가장 덜 더운날이 언제야?",
    "2주전 우리반과 옆반 합쳐서 설정온도가 가장 낮은날이 언제야?",
    "우리반의 현재 설정온도 알려줘",
    "8일전 설정온도는?",
    "10년 전 오늘 우리반 실내온도는?",
    "우리반, 옆반, 앞반 중 가장 추운 방은?",
    "지난달 오늘 오후 2시에 옆반의 설정온도는 어땠어?",
    "옆반의 현재 실내온도랑 설정온도 알려줘",
    "이번주 우리반 평균 실내온도 알려줘",
    "지난달 설정온도 평균을 알려줘.",
    "우리반 이번달 제일 추웠던 날은 언제냐?",
    "작년 옆반 가장 더웠던 달은?",
    "오늘 오전 11시에 옆반의 실내온도는 어땠어?",
    "오늘 오후 4시부터 6시까지 실내온도 평균 알려줘",
    "지난주에 설정온도와 실내온도 차이가 가장 많이 났던 날은?",
    "우리반과 옆반중 더 추운곳은 어디야?",
    "이번주 우리반과 앞반의 평균 실내온도 알려줘",
    "현재 설정온도랑 실내온도 차이 알려줘.",
    "지난달에 설정온도와 실내온도 차이가 가장 많이 났던 날은?",
    "이번주 우리반과 옆반의 평균 실내온도 차이 알려줘",
    "2주전 가장 더웠던 날 알려줘",
    "옆반 습도 알려줘",
    "지난 3일 동안 우리반 실내 실내온도 평균 값 알려줘.",
    "오늘 오후 5시에 옆반의 설정온도는 어땠어?",
    "올해 여름 우리반 실내온도 최대값과 최소값 알려줘",
    "우리반과 앞반 중 가장 더운 방은?",
]


In [ ]:
report = {
    "success": 0,
    "error": 0,
    "nan": 0,
}
for query in queries:
    result = generate(query)
    result = result.replace("CURRENT_DATE", "TIMESTAMP '2022-09-30 12:00:00'")
    try:
        results = DBManager.execute_sql(result)
        has_nan = False
        for r in results:
            # r: tuple
            # see if nan in r
            if any(pd.isna(v) for v in r):
                has_nan = True
                break
        if has_nan:
            report["nan"] += 1
        else:
            print(f"Query: {query}\nResult: {result}\n")
            print("results", results)
            report["success"] += 1
    except Exception as e:
        print(f"Error: {e}")
        report["error"] += 1

print(report)